In [1]:
import yaml
import os
import logging
import plotly
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
if not log.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    ch.setFormatter(logging.Formatter('%(levelname)s - %(message)s'))
    log.addHandler(ch)

In [2]:
import pandas as pd
outfilename = 'output.csv'
df = pd.read_csv('../../StatisticsVIO.csv', header=None, index_col=False).T
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))

In [3]:
import plotly.graph_objects as go
import numpy as np

# Rename cols
df.rename(columns={"IMU Preintegration Timing [us]": "IMU Preint. [us]",
                   "StereoFrontEnd Timing [ms]": "Frontend [ms]",
                   "Pipeline Overall Timing [ms]": "Total [ms]",
                   "Visualizer Timing [ms]": "Visualizer [ms]",
                   "Mesher Timing [ms]": "Mesher [ms]",
                   "Backend Timing [ms]": "Backend [ms]"}, inplace=True)

# Reorder cols as convenient, and drop Visualizer timing:
df = df[['IMU Preint. [us]', 'Frontend [ms]', 'Mesher [ms]', 'Backend [ms]', 'Total [ms]']]

x_data = list(df.columns.values)
y_data =[]
for column in df:
    y_data.append(df[column].values.astype(np.float))

In [49]:


colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
          'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)']

fig = go.Figure()

for xd, yd in zip(x_data, y_data):
    fig.add_trace(go.Box(
        y=yd,
        name=xd,
        boxmean=True, # represent mean and standard deviation
        jitter=0.5,
        pointpos=0,
        whiskerwidth=0.8,
        #fillcolor=cls,
        marker_size=3,
        line_width=1,
        showlegend=False,
       boxpoints='all' # only suspected outliers)
    ))
    
    fig.update_layout(
    title='Points Scored by the Top 9 Scoring NBA Players in 2012',
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)
    
fig.update_layout(
    title=go.layout.Title(
        text="Per Module Timing"
    ),
    xaxis=go.layout.XAxis(
        dtick=0.1,
        title=go.layout.xaxis.Title(
            text="Modules"
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Time"
            ),
        rangemode='tozero'
        ),
    template='plotly_white'
)
fig.update_yaxes(range=[0, 140])
fig.show()

In [37]:
fig.write_image("vio_timing.eps")